### TAO remote client (object detection with DetectNet_v2)

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### The workflow in a nutshell

- Creating a dataset
- Upload kitti dataset to the service
- Running dataset convert
- Getting a PTM from NGC
- Model Actions
    - Train
    - Evaluate
    - Prune, retrain
    - Export
    - Convert
    - Inference on TAO
    - Inference on TRT

### Table of contents

1. [Install TAO remote client ](#head-1)
1. [Set the remote service base URL](#head-2)
1. [Access the shared volume](#head-3)
1. [Create the datasets](#head-4)
1. [List datasets](#head-5)
1. [Provide and customize dataset convert specs](#head-6)
1. [Run dataset convert](#head-7)
1. [Create a model experiment](#head-8)
1. [Find detectnet_v2 pretrained model](#head-9)
1. [Customize model metadata](#head-10)
1. [Provide train specs](#head-11)
1. [Run train](#head-12)
1. [Provide evaluate specs](#head-13)
1. [Run evaluate](#head-14)
1. [Provide prune specs](#head-15)
1. [Run prune](#head-16)
1. [Provide retrain specs](#head-17)
1. [Run retrain](#head-18)
1. [Run evaluate on retrain](#head-18-1)
1. [Provide FP32 export specs](#head-19)
1. [Run FP32 export](#head-20)
1. [Provide Int8 export specs](#head-21)
1. [Run Int8 export](#head-22)
1. [Provide model convert specs](#head-23)
1. [Run model convert](#head-24)
1. [Provide TAO inference specs](#head-25)
1. [Run TAO inference](#head-26)
1. [Provide TRT inference specs](#head-27)
1. [Run TRT inference](#head-28)
1. [Delete experiment](#head-29)
1. [Delete datasets](#head-30)
1. [Unmount shared volume](#head-31)
1. [Uninstall TAO Remote Client](#head-32)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import os
import glob
import subprocess
import getpass
import uuid
import json

In [ ]:
namespace = 'default'

### Install TAO remote client <a class="anchor" id="head-1"></a>

In [ ]:
# SKIP this step IF you have already installed the TAO-Client wheel.
! pip3 install nvidia-tao-client

In [ ]:
# View the version of the TAO-Client
! tao-client --version

### FIXME

1. Assign the ip_address and port_number in FIXME 1 and FIXME 2 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
2. Set NGC API key in FIXME 3
3. Set KITTI Images and Labels URL in FIXME 4 and FIXME 5

### Set the remote service base URL and Token <a class="anchor" id="head-2"></a>

In [ ]:
# Define the node_addr and port number
node_addr = "<ip_address>" # FIXME1 example: 10.137.149.22
node_port = "<port_number>" # FIXME2 example: 32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
%env BASE_URL=http://{node_addr}:{node_port}/{namespace}/api/v1

In [ ]:
# FIXME: Set ngc_api_key valiable
ngc_api_key = "<ngc_api_key>" # FIXME3 example: zZYtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGyM

# Exchange NGC_API_KEY for JWT
identity = json.loads(subprocess.getoutput(f'tao-client login --ngc-api-key {ngc_api_key}'))

%env USER={identity['user_id']}
%env TOKEN={identity['token']}

### Access the shared volume <a class="anchor" id="head-3"></a>

In [ ]:
# Get PVC ID
pvc_id = subprocess.getoutput(f'kubectl get pvc tao-toolkit-api-pvc -n {namespace} -o jsonpath="{{.spec.volumeName}}"')
print(pvc_id)

In [ ]:
# Get NFS server info
provisioner = json.loads(subprocess.getoutput(f'helm get values nfs-subdir-external-provisioner -o json'))
nfs_server = provisioner['nfs']['server']
nfs_path = provisioner['nfs']['path']
print(nfs_server, nfs_path)

In [ ]:
user = getpass.getuser()
home = os.path.expanduser('~')

! echo "Password for {user}"
password = getpass.getpass()

In [ ]:
# Mount shared volume 
! mkdir -p ~/shared

command = "apt-get -y install nfs-common >> /dev/null"
! echo {password} | sudo -S -k {command}

command = f"mount -t nfs {nfs_server}:{nfs_path}/{namespace}-tao-toolkit-api-pvc-{pvc_id} ~/shared"
! echo {password} | sudo -S -k {command} && echo DONE

### Create the datasets <a class="anchor" id="head-4"></a>

We will be using the kitti object detection dataset for this example. To find more details, please visit [here](http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d). One can request the images from [here](http://www.cvlibs.net/download.php?file=data_object_image_2.zip), and the training labels from [here](http://www.cvlibs.net/download.php?file=data_object_label_2.zip).

**If using custom dataset; it should follow this dataset structure**
```
training
├── image_2
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   ├── ...
└── label_2
    ├── image_name_1.txt
    ├── image_name_2.txt
    ├── ...
test_samples
├── image_2
    ├── image_name_3.jpg
    ├── image_name_4.jpg
    ├── ...
evaluating
├── image_2
│   ├── image_name_5.jpg
│   ├── image_name_6.jpg
|   ├── ...
└── label_2
    ├── image_name_5.txt
    ├── image_name_6.txt
    ├── ...
```
The file name should be same for images and labels folders

In [ ]:
%env IMAGES_URL=https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip # FIXME4
%env LABELS_URL=https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip # FIXME5

! wget -O images.zip $IMAGES_URL
! wget -O labels.zip $LABELS_URL

! unzip -u images.zip
! unzip -u labels.zip

! mkdir -p evaluating/image_2 && mv training/image_2/000*.png evaluating/image_2/
! mkdir -p evaluating/label_2 && mv training/label_2/000*.txt evaluating/label_2/

! mkdir -p test_samples/image_2 && mv testing/image_2/00000*.png test_samples/image_2/

In [ ]:
train_dataset_id = subprocess.getoutput("tao-client detectnet-v2 dataset-create --dataset_type object_detection --dataset_format kitti")
print(train_dataset_id)

In [ ]:
! rsync -ah --info=progress2 training/image_2/ ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/images
! rsync -ah --info=progress2 training/label_2/ ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/labels
! echo DONE

In [ ]:
eval_dataset_id = subprocess.getoutput("tao-client detectnet-v2 dataset-create --dataset_type object_detection --dataset_format kitti")
print(eval_dataset_id)

In [ ]:
! rsync -ah --info=progress2 evaluating/image_2/ ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/images
! rsync -ah --info=progress2 evaluating/label_2/ ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/labels
! echo DONE

In [ ]:
infer_dataset_id = subprocess.getoutput("tao-client detectnet-v2 dataset-create --dataset_type object_detection --dataset_format raw")
print(infer_dataset_id)

In [ ]:
! rsync -ah --info=progress2 test_samples/image_2/ ~/shared/users/{os.environ['USER']}/datasets/{infer_dataset_id}/images
! echo DONE

### List datasets <a class="anchor" id="head-5"></a>

In [ ]:
pattern = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', '*', 'metadata.json')

datasets = []
for metadata_path in glob.glob(pattern):
    with open(metadata_path, 'r') as metadata_file:
        datasets.append(json.load(metadata_file))

print(json.dumps(datasets, indent=2))

### Provide and customize dataset convert specs <a class="anchor" id="head-6"></a>

In [ ]:
# Default train dataset specs
! tao-client detectnet-v2 dataset-convert-defaults --id {train_dataset_id} --action convert | tee ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}/specs/convert.json

In [ ]:
# Customize train dataset specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', train_dataset_id, 'specs', 'convert.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["kitti_config"]["image_extension"] = ".png"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

In [ ]:
# Default eval dataset specs
! tao-client detectnet-v2 dataset-convert-defaults --id {eval_dataset_id} --action convert | tee ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}/specs/convert.json

In [ ]:
# Customize eval dataset specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', eval_dataset_id, 'specs', 'convert.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["kitti_config"]["image_extension"] = ".png"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run dataset convert <a class="anchor" id="head-7"></a>

In [ ]:
train_convert_job_id = subprocess.getoutput(f"tao-client detectnet-v2 dataset-convert --id {train_dataset_id}  --action convert ")
print(train_convert_job_id)

In [ ]:
def my_tail(logs_dir, log_file):
    %env LOG_FILE={logs_dir}/{log_file}
    ! mkdir -p {logs_dir}
    ! [ ! -f "$LOG_FILE" ] && touch $LOG_FILE && chmod 666 $LOG_FILE
    ! tail -f -n +1 $LOG_FILE | while read LINE; do echo "$LINE"; [[ "$LINE" == "EOF" ]] && pkill -P $$ tail; done
    
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', train_dataset_id, 'logs')
log_file = f"{train_convert_job_id}.txt"

my_tail(logs_dir, log_file)

In [ ]:
eval_convert_job_id = subprocess.getoutput(f"tao-client detectnet-v2 dataset-convert --id {eval_dataset_id}  --action convert ")
print(eval_convert_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'datasets', eval_dataset_id, 'logs')
log_file = f"{eval_convert_job_id}.txt"

my_tail(logs_dir, log_file)

### Create a model experiment <a class="anchor" id="head-8"></a>

In [ ]:
model_id = subprocess.getoutput("tao-client detectnet-v2 model-create --network_arch detectnet_v2 --encryption_key tlt_encode ")
print(model_id)

### Find detectnet_v2 pretrained model <a class="anchor" id="head-9"></a>

In [ ]:
pattern = os.path.join(home, 'shared', 'users', '*', 'models', '*', 'metadata.json')

ptm_id = None
for metadata_path in glob.glob(pattern):
  with open(metadata_path, 'r') as metadata_file:
    metadata = json.load(metadata_file)
    ngc_path = metadata.get("ngc_path")
    if ngc_path and "detectnet_v2:resnet18" in ngc_path:
      ptm_id = metadata["id"]
      break

print(ptm_id)

### Customize model metadata <a class="anchor" id="head-10"></a>

In [ ]:
metadata_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'metadata.json')

with open(metadata_path , "r") as metadata_file:
    metadata = json.load(metadata_file)

metadata["train_datasets"] = [train_dataset_id]
metadata["eval_dataset"] = eval_dataset_id
metadata["inference_dataset"] = infer_dataset_id
metadata["ptm"] = ptm_id

with open(metadata_path, "w") as metadata_file:
    json.dump(metadata, metadata_file, indent=2)

print(json.dumps(metadata, indent=2))

### Provide train specs <a class="anchor" id="head-11"></a>

In [ ]:
# Default train model specs
! tao-client detectnet-v2 model-train-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/train.json

In [ ]:
# Customize train model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'train.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["training_config"]["num_epochs"] = 11

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run train <a class="anchor" id="head-12"></a>

In [ ]:
train_job_id = subprocess.getoutput("tao-client detectnet-v2 model-train --id " + model_id)
print(train_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
logs_dir = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'logs')
log_file = f"{train_job_id}.txt"

my_tail(logs_dir, log_file)

### Provide evaluate specs <a class="anchor" id="head-13"></a>

In [ ]:
# Default evaluate model specs
! tao-client detectnet-v2 model-evaluate-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/evaluate.json

In [ ]:
# Customize evaluate model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'evaluate.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["dataset_config"]["image_extension"] = "png"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run evaluate <a class="anchor" id="head-14"></a>

In [ ]:
eval_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-evaluate --id {model_id} --job {train_job_id}")
print(eval_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{eval_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide prune specs <a class="anchor" id="head-15"></a>

In [ ]:
# Default prune model specs
! tao-client detectnet-v2 model-prune-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/prune.json

### Run prune <a class="anchor" id="head-16"></a>

In [ ]:
prune_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-prune --id {model_id} --job {train_job_id}")
print(prune_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{prune_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide retrain specs <a class="anchor" id="head-17"></a>

In [ ]:
# Default retrain model specs
! tao-client detectnet-v2 model-retrain-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/retrain.json

In [ ]:
# Customize retrain model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'retrain.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["training_config"]["num_epochs"] = 10
specs["dataset_config"]["image_extension"] = "png"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run retrain <a class="anchor" id="head-18"></a>

In [ ]:
retrain_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-retrain --id {model_id} --job {prune_job_id}")
print(retrain_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{retrain_job_id}.txt"
my_tail(logs_dir, log_file)

### Run evaluate on retrained model <a class="anchor" id="head-18-1"></a>

In [ ]:
eval2_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-evaluate --id {model_id} --job {retrain_job_id}")
print(eval2_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{eval2_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide FP32 export specs <a class="anchor" id="head-19"></a>

In [ ]:
# Default export model specs
! tao-client detectnet-v2 model-export-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/export.json

In [ ]:
# Customize export model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'export.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["data_type"] = "fp32"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run FP32 export <a class="anchor" id="head-20"></a>

In [ ]:
fp32_export_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-export --id {model_id} --job {train_job_id}")
print(fp32_export_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{fp32_export_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide Int8 export specs <a class="anchor" id="head-21"></a>

In [ ]:
# Default export model specs
! tao-client detectnet-v2 model-export-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/export.json

In [ ]:
# Customize export model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'export.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["data_type"] = "int8"
specs["batches"] = 10
specs["batch_size"] = 4
specs["max_batch_size"] = 4

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run Int8 export <a class="anchor" id="head-22"></a>

In [ ]:
int8_export_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-export --id {model_id} --job {train_job_id}")
print(int8_export_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{int8_export_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide model convert specs <a class="anchor" id="head-23"></a>

In [ ]:
# Default convert model specs
! tao-client detectnet-v2 model-convert-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/convert.json

In [ ]:
# Customize convert model specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'convert.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["t"] = "int8"
specs["b"] = 10
specs["m"] = 64
specs["d"] = "3,384,1248"
specs["o"] = "output_cov/Sigmoid,output_bbox/BiasAdd"

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run model convert <a class="anchor" id="head-24"></a>

In [ ]:
convert_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-convert --id {model_id} --job {int8_export_job_id}")
print(convert_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{convert_job_id}.txt"
my_tail(logs_dir, log_file)

### Provide TAO inference specs <a class="anchor" id="head-25"></a>

In [ ]:
# Default inference model specs
! tao-client detectnet-v2 model-inference-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/inference.json

In [ ]:
# Customize TAO inference specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'inference.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["inferencer_config"]["batch_size"] = 4

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TAO inference <a class="anchor" id="head-26"></a>

In [ ]:
tlt_inference_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-inference --id {model_id} --job {train_job_id}")
print(tlt_inference_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{tlt_inference_job_id}.txt"
my_tail(logs_dir, log_file)

In [ ]:
from IPython.display import Image
import glob
job_dir = f"{home}/shared/users/{os.environ['USER']}/models/{model_id}/{tlt_inference_job_id}"
sample_image = glob.glob(f"{job_dir}/images_annotated/*.png")[6]
Image(filename=sample_image) 

### Provide TRT inference specs <a class="anchor" id="head-27"></a>

In [ ]:
# Default inference model specs
! tao-client detectnet-v2 model-inference-defaults --id {model_id} | tee ~/shared/users/{os.environ['USER']}/models/{model_id}/specs/inference.json

In [ ]:
# Customize TAO inference specs
specs_path = os.path.join(home, 'shared', 'users', os.environ['USER'], 'models', model_id, 'specs', 'inference.json')

with open(specs_path , "r") as specs_file:
    specs = json.load(specs_file)

specs["inferencer_config"]["batch_size"] = 4

with open(specs_path, "w") as specs_file:
    json.dump(specs, specs_file, indent=2)

print(json.dumps(specs, indent=2))

### Run TRT inference <a class="anchor" id="head-28"></a>

In [ ]:
trt_inference_job_id = subprocess.getoutput(f"tao-client detectnet-v2 model-inference --id {model_id} --job {int8_export_job_id}")
print(trt_inference_job_id)

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
log_file = f"{trt_inference_job_id}.txt"
my_tail(logs_dir, log_file)

### Delete experiment <a class="anchor" id="head-29"></a>

In [ ]:
! rm -rf ~/shared/users/{os.environ['USER']}/models/{model_id}
! echo DONE

### Delete datasets <a class="anchor" id="head-30"></a>

In [ ]:
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{train_dataset_id}
! rm -rf ~/shared/users/{os.environ['USER']}/datasets/{eval_dataset_id}
! echo DONE

### Unmount shared volume <a class="anchor" id="head-31"></a>

In [ ]:
command = "umount ~/shared"
! echo {password} | sudo -S -k {command} && echo DONE

### Uninstall TAO Remote Client <a class="anchor" id="head-32"></a>

In [ ]:
! pip3 uninstall -y nvidia-tao-client